In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import json
import re

url = 'https://www.d20pfsrd.com/classes/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'base-classes/oracle/'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        self.href2_list = []



    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        result = re.sub(r'<.*?>', '', result)        

        return result

    def link_searcher(self, a_tags):
        for a in a_tags:
            # Check if the link contains "investigator-talents" in the href attribute
            if "oracle-mysteries" in str(a.get('href', '')):
                # Extract everything within double quotes in the href attribute
                regex = r'href="(.*?)"'
                href_unclean = str(a)
                match = re.search(regex, href_unclean)

                if match:
                    self.href2 = match.group(1)
                    self.href2_list.append(self.href2)
                else:
                    continue
            else:
                continue    


        return self.href2_list

         

# What we want to do is to get the data once we see a <b> tag, use the <b> tag as the key section
# Since it's going to be the same each time we can manually set them (like in the items json file)
    def div_search(self, td_tags):
        
        result_list = []

        for td in td_tags:
            a_tags = td.find_all('a')
            data_dict = {}            
            href2_list = self.link_searcher(a_tags)

        
    def talent_info_search(self):
        # Initialize an empty list to store dictionaries for each iteration
        all_data_dicts = []

        for url2 in self.href2_list:
            self.detail_page = requests.get(url2)
            self.detail_soup = BeautifulSoup(self.detail_page.content, 'html.parser')

            div_tags = self.detail_soup.find_all('div', class_="article-content")
            b_tags = self.detail_soup.find_all('b')

            for div in div_tags:
                p_tags = div.find_all('p')
                h_tag = div.find('h1')

                data_dict = {}
                data_dict[h_tag.get_text()] = {}

                for p_tag in p_tags:
                    # Split text at ':' to get key-value pairs
                    p_tag_clean = self.remove_parenthesis(p_tag.get_text())
                    parts = p_tag_clean.split(':')
                    if len(parts) > 1:
                        key = self.remove_parenthesis(parts[0].strip())
                        value = self.remove_parenthesis(parts[1].strip())
                        data_dict[h_tag.get_text()][key] = value

                all_data_dicts.append(data_dict)

        # Convert the list of dictionaries to a JSON string
        json_data = json.dumps(all_data_dicts, indent=2)

        # Print or return the JSON string as needed
        print(json_data)
        return json_data



    def main(self):        
        td_tags = self.soup.find_all('td')
        self.div_search(td_tags)
        self.talent_info_search()


# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)

# Call the get_class_info method
scraper_instance.main()

[]


In [60]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import json
import re

url = 'https://www.aonprd.com/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'OracleCurses.aspx'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        self.href2_list = []



    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        result = re.sub(r'<.*?>', '', result)        

        return result

# What we want to do is to get the data once we see a <b> tag, use the <b> tag as the key section
# Since it's going to be the same each time we can manually set them (like in the items json file)
    def div_search(self, td_tags):
        
        result_list = []

        for td in td_tags:
            a_tags = td.find_all('a')
            data_dict = {}            
            href2_list = self.link_searcher(a_tags)

        
    def talent_info_search(self, td_tags):
        # Initialize an empty list to store dictionaries for each iteration
        all_data_dicts = []
        current_level = None
        current_text = ""

        for td in td_tags:
            td_text = td.get_text(strip=True)
            clean_td = self.remove_parenthesis(td_text)

            # Check if the current text contains a level indicator
            if "5th level" in clean_td or "10th level" in clean_td or "15th level" in clean_td:
                # Save the previous data (if any)
                if current_level is not None:
                    all_data_dicts.append({current_level: current_text})

                # Update the current level and reset the text
                current_level = clean_td
                current_text = ""
            else:
                # Append the current text to the ongoing content
                current_text += clean_td

        # Save the last set of data
        if current_level is not None:
            all_data_dicts.append({current_level: current_text})

        # Convert the list of dictionaries to a JSON string
        json_data = json.dumps(all_data_dicts, indent=2)
        print(json_data)
        return json_data



    def main(self):        
        td_tags = self.soup.find_all('td')
        # self.div_search(td_tags)
        self.talent_info_search(td_tags)


# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)

# Call the get_class_info method
scraper_instance.main()

[
  {
    "Aboleth: Your mind is marked by aboleth tampering, either as a result of a close encounter with one of these creatures while you were a child, or even through an ancestor, close family member, or one of your sisters or brothers in faith\u2014this associate\u2019s interaction with the aboleths may be all that it takes to infect you. You take a \u20132 penalty on saving throws against mind-affecting effects and addcharm personandhypnotismto your list of 1st-level oracle spells known. At 5th level, addminor imageto your list of 2nd-level oracle spells known. At 10th level, addmodify memoryto your list of 4th-level oracle spells known. At 15th level, addveilto your list of 6th-level oracle spells known.": ""
  },
  {
    "Acursed: You are cursed with misfortune and sorrow, and you cannot gain benefit from morale bonuses. However, you gain a +4 bonus to all saving throws against curse effects. At 5th level, addill omenAPGto your list of spells known. At 10th level, addgreater bra

In [2]:
from bs4 import BeautifulSoup
import requests
import json

url = 'https://www.aonprd.com/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = 'OracleMysteries.aspx'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')

    def clean_text(self, text):
        # Clean and normalize text
        return ' '.join(text.split()).replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"')

    def parse_mystery(self):
        # Find the main container
        container = self.soup.find('span', id=re.compile(r"MainContent_DataListTypes_LabelName_\d+"))
        if not container:
            print("No matching container found.")
            return

        # Extract the title
        title_tag = container.find('h1')
        if title_tag:
            title = title_tag.get_text(strip=True)
            print(f"Processing mystery: {title}")
            data_dict = {title: {}}
        else:
            print("No title found.")
            return

        # Extract content under the container
        current_key = None
        for element in container.find_all(['b', 'i', 'br'], recursive=False):
            if element.name == 'b':
                # Capture the key
                current_key = element.get_text(strip=True).replace(":", "")
                data_dict[title][current_key] = ""
            elif element.name == 'br' and current_key:
                # Append the following text as part of the value
                next_sibling = element.next_sibling
                if next_sibling:
                    value = self.clean_text(next_sibling.get_text(strip=True) if next_sibling.name else str(next_sibling))
                    data_dict[title][current_key] += value
            elif element.name == 'i' and current_key:
                # Add italicized text to the current value
                value = self.clean_text(element.get_text(strip=True))
                data_dict[title][current_key] += " " + value

        # Convert the dictionary to JSON for readability
        json_data = json.dumps(data_dict, indent=2)
        print(json_data)
        return json_data

    def main(self):
        self.parse_mystery()

# Instantiate and run the scraper
scraper_instance = ArchetypeScraper(url)
scraper_instance.main()


No matching container found.
